# Parsing du document texte

In [101]:
import codecs
import pandas as pd


# Ouvrir le fichier en lecture seule
file = codecs.open('Planning_JO2024_v3.txt', "r", "utf-8")
# utiliser readlines pour lire toutes les lignes du fichier
# La variable "lignes" est une liste contenant toutes les lignes du fichier
lines = file.readlines()
# fermez le fichier après avoir lu les lignes
file.close()


words = []
for line in lines:
    words += line.split()
    
demarq1_discipline = "calendrier reste susceptible de modification jusqu'à la conclusion des Jeux Olympiques.".split()
demarq2_discipline = "A noter que l'ordre des matchs sera déterminé pendant les Jeux en fonction du temps de repos des athlètes, des demandes des diffuseurs et d'autres paramètres qu'il n'est pas possible d'anticip".split()
discipline = 'ATHLETISME'

data = []
i = 0
while i < (len(words)):
    word = words[i]        
    if words[i].upper() == "JOUR":
        if len(words) > i + 4:
            jour = words[i+1]
            date = words[i+2] + " " + words[i+3] + " " + words[i+4]
            i +=4
        else:
            erreur.append(line)
    
    # discipline
    if words[i:i+len(demarq1_discipline)] == demarq1_discipline:
        i+=len(demarq1_discipline)
        if words[i:i+len(demarq2_discipline)] == demarq2_discipline:
            i+=len(demarq2_discipline)
        discipline = ""
        espace = ""
        while words[i] != "Calendrier":
            discipline += espace + words[i]
            espace = " "
            i += 1
    
    if words[i] == "Session":
        session = words[i+1]
        i += 2
    if words[i] == "Début":
        debut = words[i+2]
        i += 3
    if words[i] == "Fin":
        fin = words[i+2]
        i += 3
        lieu_epreuve = ""
        espace = ""
        while words[i] != "Nom":
            lieu_epreuve += espace + words[i]
            espace = " "
            i += 1
    nom_epreuve = ""
    if words[i:i+4] == ['Nom', 'de', "l'épreuve/des", 'épreuves']:
        i += 4
        espace = ""
        while len(words) > i and words[i].upper() not in ["JOUR", "SESSION"] and words[i:i+2] != ['Le', 'calendrier']:
            nom_epreuve += espace + words[i]
            espace = " "
            i += 1
        data.append([jour, date, discipline, session, debut, fin, lieu_epreuve, nom_epreuve])
    i += 1

def traitement_discipline(string):
    if len(string) == 0:
        return ""
    else:
        return string[0].upper() + string[1:].lower()


df = pd.DataFrame(data, columns=['num_jour', 'date', 'discipline', 'num_session', 'debut_epreuve', 'fin_epreuve', 'lieu_epreuve', 'nom_epreuve'])
df['discipline'] = df['discipline'].apply(traitement_discipline)
#df.to_csv('../data/session_planning_olympics.csv')

# Parsing noms d'epreuves

In [7]:
df["nom_epreuve"][0]

'20 km marche - hommes, finale 20 km marche - femmes, finale'

In [85]:
texte = '20 km marche - hommes, finale 20 km marche - femmes, finale'
separateurs = [' - ', ', '], 
noms = ['epreuve', 'genre', 'etape']

parser(texte, separateurs, noms)

'{"content": [{"etape": "20 km marche - hommes, finale 20 km marche - femmes, finale"}]}'

In [17]:
def nom_epreuve(texte, i):
    for nom in noms_sports:
        if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
            return True
    return False


In [102]:
import json

# separateurs = [' - ', ', '], 
# noms = ['epreuve', 'genre', 'etape']


def parser(texte, separateurs, noms):
    #print(texte)
    epreuves = {'content' : []}
    # epreuves = []
    # epreuve, genre, etape = '', '', ''
    dico_epreuve = {}
    i, i0 = 0, 0
    while i < len(texte):
        #print(i)
        for n in range(len(separateurs)):
            separateur = separateurs[n]
            #print("Chgt separateur : ", separateur)
            while i < len(texte):
                if i + len(separateur) < len(texte) and texte[i:i+len(separateur)] == separateur:
                    dico_epreuve[noms[n]] = texte[i0:i]
                    #print(texte[i0:i])
                    i+=len(separateur)
                    i0 = i
                    break
                i+=1
        #print(i0)
        while i <= len(texte):
            if  i == len(texte) or nom_epreuve(texte, i):
                dico_epreuve[noms[-1]] = texte[i0:i]
                i0 = i
                break
            i+=1
        epreuves['content'].append(dico_epreuve.copy())#json.dumps(dico_epreuve.copy()))
        # epreuves.append(dico_epreuve.copy())
        i+=1
    return json.dumps(epreuves)

noms_sports = [
    "20 km",
    "100 m",
    "200 m",
    "1 500 m",
    "5 000 m",
    "800 m",
    "10 000 m",
    "3 000 m",
    "400 m",
    "110 m",
    "50kg",
    "51kg",
    "54kg",
    "60kg",
    "66kg",
    "75kg",
    "63.5kg",
    "63,5kg",
    "80kg",
    "92kg",
    "71kg",
    "57kg",
    "+92kg"
]

pas_noms_epreuve = [
    "Femmes",
    "Hommes",
    "Repêchage",
    "Qualifications",
    "Eliminatoires",
    "Quart de finale",
    "Demi-finale",
    "Match pour la médaille",
    "Série",
    "Course pour"
]

# inclure pas nom d'épreuve
def nom_epreuve(texte, i):
    if texte[i].isupper():
        for nom in pas_noms_epreuve:
            if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
                return False            
        if i + 3 < len(texte) and texte[i+1] != " " and texte[i+2] != " " and texte[i+3] != " " :
            return True
    for nom in noms_sports:
        if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
            return True
    return False

noms_etapes = ["eliminatoires", "repêchage"] # --> enlever majuscule

# Rajouter condition sur nom sport: plus de 3 lettres
# pour tej les "finale A/B" et autres

cles_parsers = {
    'ATH ROW BMF CTR CLB FEN ARC SHO BOX CSL CSP WRE JUD SWM' : {
            'separateurs' : [' - ', ', '], 
            'noms' : ['epreuve', 'genre', 'etape']
    },
    'BKB BMX BK3 RU7 BKG GTR HBL HOC SRF TTE VVO WPO' : {
        'separateurs' : [', '], 
        'noms' : ['genre', 'etape']
    },
    'CRD OWS DIV MTB WLF' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'genre']
    },
    'GLF MPN SAL' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'etape']
    },
    'FBL' : {
        'separateurs' : [', ', ' - '], 
        'noms' : ['genre', 'etape', 'epreuve']
    },
    'GAR VBV' : {
        'separateurs' : [' - '], 
        'noms' : ['etape', 'genre']
    },
    'GRY BDM' : {
        'separateurs' : [', '], 
        'noms' : ['epreuve', 'etape']
    },
    'SWA EQU TKW TEN SKB' : {
        'separateurs' : [], 
        'noms' : ['epreuve']
    },
    'WLF TRI' : {
        'separateurs' : [], 
        'noms' : ['genre']
    },
}


def match_cle_parser(code_disc, cles_parsers):
    for cle in cles_parsers.keys():
        if code_disc in cle:
            return cles_parsers[cle]
    print(f"Cette discipline n'est pas prise en charge : {code_disc}")

def parsing_epreuve(row):
    texte = row.nom_epreuve
    if len(str(row.num_session)) > 2:
        disc = str(row.num_session)[:3]
        cle_parser = match_cle_parser(disc, cles_parsers)
        if cle_parser is not None:
            return parser(texte, cle_parser['separateurs'], cle_parser['noms'])

# disc_analysees = ['ATHLETISME', 'AVIRON', 'BASKETBALL', 'BASKETBALL 3x3', 'BMX FREESTYLE', 'BMX RACING', 'Breaking']

df['parsing_epreuve'] = df.apply(parsing_epreuve, axis=1)
df.to_csv('../data/session_planning_olympics_v10.csv')
# df

# Bug fix

In [90]:
df.loc[df["discipline"] == "Rugby"]

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve
460,-2,Mer 24 Juillet,Rugby,RU701,15:00,17:00,Centre aquatique,"Hommes, poules (12 matchs)","{""content"": [{""genre"": ""Hommes"", ""etape"": ""pou..."
461,-2,Mer 24 Juillet,Rugby,RU702,15:00,17:00,Centre aquatique,"Hommes, poules (6 matchs)","{""content"": [{""genre"": ""Hommes"", ""etape"": ""pou..."
462,-2,Mer 24 Juillet,Rugby,RU702,15:00,17:00,Centre aquatique,"Hommes, placement 9-12 (2 matchs) Hommes, quar...","{""content"": [{""genre"": ""Hommes"", ""etape"": ""pla..."
463,-2,Mer 24 Juillet,Rugby,RU702,15:00,17:00,Centre aquatique,"Hommes, placement 5-8 (2 matchs) Hommes, demi-...","{""content"": [{""genre"": ""Hommes"", ""etape"": ""pla..."
464,-2,Mer 24 Juillet,Rugby,RU705,15:00,17:00,Centre aquatique,"Femmes, poules (12 matchs)","{""content"": [{""genre"": ""Femmes"", ""etape"": ""pou..."
465,-2,Mer 24 Juillet,Rugby,RU706,15:00,17:00,Centre aquatique,"Femmes, poules (6 matchs)","{""content"": [{""genre"": ""Femmes"", ""etape"": ""pou..."
466,-2,Mer 24 Juillet,Rugby,RU706,15:00,17:00,Centre aquatique,"Femmes, placement 9-12 (2 matchs) Femmes, quar...","{""content"": [{""genre"": ""Femmes"", ""etape"": ""pla..."
467,-2,Mer 24 Juillet,Rugby,RU706,15:00,17:00,Centre aquatique,"Femmes, placement 5-8 (2 matchs) Femmes, demi-...","{""content"": [{""genre"": ""Femmes"", ""etape"": ""pla..."


In [82]:
df.loc[df["discipline"] == "Beach volleyball", "nom_epreuve"][624]

'Tour préliminaire - Femmes et hommes (2 matchs)'

In [87]:
# disciplines ou le ", " ne marche pas comme séparateur
# CRD BKB 

for i in range(len(df)):
    print(df['discipline'][i], df['parsing_epreuve'][i]) #["content"]

Athletisme {"content": [{"epreuve": "20 km marche", "genre": "hommes", "etape": "finale "}, {"epreuve": "20 km marche", "genre": "femmes", "etape": "finale"}]}
Athletisme {"content": [{"epreuve": "D\u00e9cathlon", "genre": "100 m", "etape": "hommes "}, {"epreuve": "Lancer de marteau", "genre": "hommes", "etape": "qualifications "}, {"epreuve": "Saut en hauteur", "genre": "femmes", "etape": "qualifications "}, {"epreuve": "100 m", "genre": "femmes", "etape": "tour pr\u00e9liminaire "}, {"epreuve": "D\u00e9cathlon", "genre": "saut en longueur", "etape": "hommes "}, {"epreuve": "1 500 m", "genre": "hommes", "etape": "1er tour "}, {"epreuve": "100 m", "genre": "femmes", "etape": "1er tour "}, {"epreuve": "D\u00e9cathlon", "genre": "lancer de poids", "etape": "hommes"}]}
Athletisme {"content": [{"epreuve": "D\u00e9cathlon", "genre": "saut en hauteur", "etape": "hommes "}, {"epreuve": "5 000 m", "genre": "femmes", "etape": "1er tour "}, {"epreuve": "Triple saut", "genre": "femmes", "etape": 

# Matching localisations

In [104]:
import pandas as pd

# import data
df1 = pd.read_csv("../data/paris-2024-sites-olympiques-et-paralympiques-franciliens.csv", delimiter=";")
# df = pd.read_csv("../data/session_planning_olympics_v9.csv")
matching_infras = pd.read_csv("../data/matching_infras.csv")
matching_infras2 = matching_infras.rename(columns={"Planning" : "lieu_epreuve", "Infra" : "NOM"})

# merge
df3 = pd.merge(df, matching_infras2, how='left', on="lieu_epreuve")
df4 = pd.merge(df3, df1, how='left', on="NOM")

# processing localisation
df4.loc[df4['Geo Point'].notna(), 'latitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[0]))
df4.loc[df4['Geo Point'].notna(), 'longitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[1]))

# filtrer les nan
df4 = df4.loc[df4['Geo Point'].notna()]
df4.to_csv('../data/session_planning_pars_with_loc_v11.csv')

df4

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve,NOM,Geo Point,SITES OLYMPIQUES PARALYMPIQUES,SPORTS,latitude,longitude
1,6,Jeudi 1 Août,Athletisme,ATH02,10:00,13:00,Stade de France,"Décathlon - 100 m, hommes Lancer de marteau - ...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
2,6,Jeudi 1 Août,Athletisme,ATH02,17:40,22:00,Stade de France,"Décathlon - saut en hauteur, hommes 5 000 m - ...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
3,6,Jeudi 1 Août,Athletisme,ATH04,10:00,16:00,Stade de France,"Décathlon - 110 m haies, hommes Saut à la perc...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
4,6,Jeudi 1 Août,Athletisme,ATH04,19:00,22:00,Stade de France,"1 500 m - hommes, Repêchages Lancer de poids -...","{""content"": [{""epreuve"": ""1 500 m"", ""genre"": ""...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
5,6,Jeudi 1 Août,Athletisme,ATH06,10:00,13:15,Stade de France,"3 000 m steeple - femmes, 1er tour Lancer de m...","{""content"": [{""epreuve"": ""3 000 m steeple"", ""g...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,1,Samedi 27 Juillet,Water polo,WPO31,18:00,21:00,Paris La Défense Arena,"Hommes, match de classement Hommes, demi-finale","{""content"": [{""genre"": ""Hommes"", ""etape"": ""mat...",La Défense Arena,"48.89566, 2.22971","Site olympique,Site paralympique","Natation,Para natation,Water-polo",48.895660,2.229710
750,1,Samedi 27 Juillet,Water polo,WPO33,09:00,12:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour...","{""content"": [{""genre"": ""Femmes"", ""etape"": ""mat...",La Défense Arena,"48.89566, 2.22971","Site olympique,Site paralympique","Natation,Para natation,Water-polo",48.895660,2.229710
751,1,Samedi 27 Juillet,Water polo,WPO33,14:00,21:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour...","{""content"": [{""genre"": ""Femmes"", ""etape"": ""mat...",La Défense Arena,"48.89566, 2.22971","Site olympique,Site paralympique","Natation,Para natation,Water-polo",48.895660,2.229710
752,1,Samedi 27 Juillet,Water polo,WPO35,09:00,12:00,Paris La Défense Arena,"Hommes, match de classement Hommes, match pour...","{""content"": [{""genre"": ""Hommes"", ""etape"": ""mat...",La Défense Arena,"48.89566, 2.22971","Site olympique,Site paralympique","Natation,Para natation,Water-polo",48.895660,2.229710


In [107]:
df4[df4["discipline"] == "Athletisme"]

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve,parsing_epreuve,NOM,Geo Point,SITES OLYMPIQUES PARALYMPIQUES,SPORTS,latitude,longitude
1,6,Jeudi 1 Août,Athletisme,ATH02,10:00,13:00,Stade de France,"Décathlon - 100 m, hommes Lancer de marteau - ...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
2,6,Jeudi 1 Août,Athletisme,ATH02,17:40,22:00,Stade de France,"Décathlon - saut en hauteur, hommes 5 000 m - ...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
3,6,Jeudi 1 Août,Athletisme,ATH04,10:00,16:00,Stade de France,"Décathlon - 110 m haies, hommes Saut à la perc...","{""content"": [{""epreuve"": ""D\u00e9cathlon"", ""ge...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
4,6,Jeudi 1 Août,Athletisme,ATH04,19:00,22:00,Stade de France,"1 500 m - hommes, Repêchages Lancer de poids -...","{""content"": [{""epreuve"": ""1 500 m"", ""genre"": ""...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
5,6,Jeudi 1 Août,Athletisme,ATH06,10:00,13:15,Stade de France,"3 000 m steeple - femmes, 1er tour Lancer de m...","{""content"": [{""epreuve"": ""3 000 m steeple"", ""g...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
6,6,Jeudi 1 Août,Athletisme,ATH06,18:30,22:00,Stade de France,"400 m - hommes, 1er tour 100 m - hommes, demi-...","{""content"": [{""epreuve"": ""400 m"", ""genre"": ""ho...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
7,6,Jeudi 1 Août,Athletisme,ATH08,10:00,13:15,Stade de France,"400 m haies - hommes, 1er tour Lancer de disqu...","{""content"": [{""epreuve"": ""400 m haies"", ""genre...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
8,6,Jeudi 1 Août,Athletisme,ATH08,18:30,22:00,Stade de France,"3 000 m steeple - hommes, 1er tour 200 m - hom...","{""content"": [{""epreuve"": ""3 000 m steeple"", ""g...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
9,6,Jeudi 1 Août,Athletisme,ATH10,10:00,13:00,Stade de France,"1 500 m - femmes, 1er tour Lancer de javelot -...","{""content"": [{""epreuve"": ""1 500 m"", ""genre"": ""...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
10,6,Jeudi 1 Août,Athletisme,ATH10,19:00,22:00,Stade de France,"400 m - hommes, demi-finales 400 m haies - fem...","{""content"": [{""epreuve"": ""400 m"", ""genre"": ""ho...",Stade de France,"48.924388, 2.359871","Site olympique,Site paralympique","Athlétisme,Para athlétisme,Rugby",48.924388,2.359871
